In [ ]:
import json
import tweepy
import requests
import botometer
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite

# FAQ

## How can I find an account's ID if I have their username?
You can use https://tweeterid.com to convert between user ids and usernames.

## If I have a user or tweet id, how can I know when it was created?
Users and tweet ids encode their creation timestamp. This tool allows you to get the datetime object from the id: https://oduwsdl.github.io/tweetedat/

## Is there an easy way to identify websites that publish content that is high or low credibility?
Scrape iffy+ to get a list of websites and their credibility ratings according to fact checkers: https://iffy.news/iffy-plus/

## Which Twitter search operators are available?
This page contains a list of the available operators: https://github.com/igorbrigadir/twitter-advanced-search

# Authentication

In [ ]:
twitter_app_auth = {
    'consumer_key': '',
    'consumer_secret': '',
    'access_token': '',
    'access_token_secret': '',
}

rapidapi_key_botometer = ""
rapidapi_key_hoaxy = ""

In [ ]:
auth = tweepy.OAuthHandler(twitter_app_auth['consumer_key'], twitter_app_auth['consumer_secret'])
auth.set_access_token(twitter_app_auth['access_token'], twitter_app_auth['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

# Hoaxy

In [ ]:
import requests

url = "https://api-hoaxy.p.rapidapi.com/top-articles"

querystring = {"most_recent":"true","exclude_tags":"[\"satire\"]"}

headers = {
    "X-RapidAPI-Host": "api-hoaxy.p.rapidapi.com",
    "X-RapidAPI-Key": rapidapi_key_hoaxy
}

response = requests.request("GET", url, headers=headers, params=querystring)

In [ ]:
for article in json.loads(response.text)['articles']:
    if article['site_type'] == 'claim': # fact_checking or claim
        print(article)

# Twitter Search

In [ ]:
results = api.search('zerohedge filter:links', count=100)

In [ ]:
results[0]._json

# Botometer

In [ ]:
uids = set([result._json['user']['id'] for result in results])

bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key_botometer,
                          **twitter_app_auth)

result = bom.check_account('@christorreslugo')
print(result)

In [ ]:
botscores = dict()
for screen_name, result in bom.check_accounts_in(list(uids)):
    botscores[screen_name] = result['cap']['universal']

# Bipartite network

In [ ]:
uids = []
urls = []
uid_url = []
for result in results:
    if result._json['entities']['urls']:
        uids.append(result._json['user']['id'])
        urls.append(result._json['entities']['urls'][0]['expanded_url'])
        uid_url.append((uids[-1], urls[-1]))

In [ ]:
B = nx.Graph()

B.add_nodes_from(uids, bipartite=0)
B.add_nodes_from(urls, bipartite=1)

B.add_edges_from(uid_url)

In [ ]:
G = bipartite.weighted_projected_graph(B, uids)

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

In [ ]:
pd.Series([edge[2]['weight'] for edge in G.edges(data=True)]).value_counts().sort_index()

In [ ]:
nx.set_node_attributes(G, botscores, "botscores")

In [ ]:
nodes_to_keep = []
for cc in nx.connected_components(G):
    if len(cc) > 5:
        nodes_to_keep += cc

In [ ]:
H = G.subgraph(nodes_to_keep)

In [ ]:
nx.write_gml(G, 'filtered_projected_bipartite.gml')

# Friends or followers networks

In [ ]:
I = nx.DiGraph()
for username in ['christorreslugo', 'osome_iu', 'ccomuprrp']:
    for friend in tweepy.Cursor(api.friends, screen_name='christorreslugo').items():
        I.add_edge(username,friend._json['screen_name'])